## Notebook for the annotation transferring from 3 donors Gut Cell Atlas to Joanito 2022 data
### Developed by: Anna Maguza

### Institute of Computational Biology - Computational Health Centre - Hemlholtz Munich

### 21 November 2022

#### Load required packages

In [1]:
import scnym

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as an

 

#### Setup Cells


In [ ]:
%matplotlib inline

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

#### Upload Data

##### Upload Control data (3 donors)

In [ ]:
input_Normal_file = '/lustre/groups/talaveralopez/workspace/anna.maguza/Processed_datasets/Gut_cell_atlas_three_donors.h5ad'  # the file that will store the analysis results
output_Normal_file = '/lustre/groups/talaveralopez/workspace/anna.maguza/Processed_datasets/Gut_cell_atlas_three_donors_output.h5ad'  # the file that will store the analysis results

In [ ]:
#Anndata upload
control_ad = sc.read_h5ad(input_Normal_file)
control_ad.X

##### Upload cancer data (Joanito-2022)

In [ ]:
#Loading again to further processing
input_CRC_file = '/lustre/groups/talaveralopez/datasets/Colorectal_cancer/Anndata/joanito_2022_anndata_raw.h5ad'  # the file that will store the analysis results
output_CRC_file = '/lustre/groups/talaveralopez/workspace/anna.maguza/Processed_datasets/joanito_2022_anndata_output.h5ad'  # the file that will store the analysis results

In [ ]:
#Anndata upload
Joanito_ad = sc.read_h5ad(input_CRC_file)
Joanito_ad.X

In [ ]:
#Quick filtering
#Filter out normal (non-tumor samles)
Joanito_ad_tumor = Joanito_ad[Joanito_ad.obs['sample.origin'] == "Tumor", :]
#Filtering by genes
Joanito_ad_tumor = Joanito_ad_tumor[Joanito_ad_tumor.obs.n_genes_by_counts < 4000, :]
Joanito_ad_tumor = Joanito_ad_tumor[Joanito_ad_tumor.obs.n_genes_by_counts > 200, :]
#Filtering by counts
Joanito_ad_tumor = Joanito_ad_tumor[Joanito_ad_tumor.obs.total_counts < 20000, :]

In [ ]:
#Creating a new column in obs
Joanito_ad_tumor.obs["category"] = "Unlabeled"
Joanito_ad_tumor.obs

#### Renaming columns that they have similar names with control dataset

In [ ]:
Joanito_ad_tumor.obs["study"] = "Covid"
Joanito_ad_tumor.obs.rename(columns={"sample_id": "sample"}, inplace=True)
Joanito_ad_tumor.obs.rename(columns={"Sex": "gender"}, inplace=True)
Joanito_ad_tumor.obs.rename(columns={"COVID_status": "status"}, inplace=True)
Joanito_ad_tumor.var.rename(columns={"name": "gene_name"}, inplace=True)

#### LogP1 normalization


In [ ]:
#Log normalization of both datasets

sc.pp.log1p(tb_healthy)
sc.pp.log1p(covid_healthy)